In [1]:
import psycopg2, os

print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host="localhost",
    port='5432',
    database="postgres",
    user="postgres",
    password="123")
conn.autocommit=True

Connecting to the PostgreSQL database...


In [2]:
# create a cursor
cur = conn.cursor()

In [3]:
import pandas as pd
queryCmd = '''
SET datestyle TO "ISO, MDY";
SELECT * FROM NEW_VIEW
WHERE RN <=3
           ;'''

cur.execute(queryCmd)  

df=pd.read_sql(queryCmd,conn)

In [4]:
print(df)

        type                County  positive14day    condition  rn
0   New_York              Herkimer       6.862000         High   1
1   New_York                Broome       6.872000         High   2
2   New_York                 Wayne       7.132667         High   3
3   New_York              Hamilton       0.734667          Low   1
4   New_York                 Bronx       1.342667          Low   2
5   New_York            Chautauqua       1.874000          Low   3
6   New_York            Livingston       5.100000  Medium High   1
7   New_York                Otsego       5.174000  Medium High   2
8   New_York        Capital Region       5.240000  Medium High   3
9   New_York              Dutchess       3.749333   Medium Low   1
10  New_York                Orange       3.760000   Medium Low   2
11  New_York              New York       3.815333   Medium Low   3
12        LA              Rosewood       0.128528         High   1
13        LA               Avocado       0.132841         High

In [5]:
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27")
db = client.apan5400
import json

with open("merge_list.json") as f:
    reviews=json.load(f)
print(len(reviews))

398622


In [6]:
#reviews

In [7]:
collection = db.airbnbs
collection.insert_many(reviews)

In [8]:
from bson.son import SON
def function(region):
    q=[

        {"$match":{ "comments":{"$regex":"good|best|great|supper|clean|nice|friendly|better|safe",
                                   "$options":'i'}, "Name" : {"$regex":region,"$options":'i'}

                  }
        },
        { "$group":{ "_id":{'listing_id': "$listing_id", 'Bedrooms':"$Bedrooms",
                            'Bathrooms':"$Bathrooms", 'Property_type': "$Property type",
                           'Airbnb_Scores':"$Review Scores Rating", 'Cleanliness':"$Review Scores Cleanliness"},
                    
                    "count": {"$sum": 1 }}},

         {"$sort": SON([("count", -1), ("_id", -1)])},
         {"$limit":5}

         ]    
    return list(collection.aggregate(q))

#function('New York')

In [9]:
import pandas as pd
from flask import Flask, request, url_for, redirect, render_template
app = Flask("Interactive App")

@app.route('/')
def home():
    return render_template("apan5400home.html")

@app.route('/', methods=['POST'])
def regionselect():
    if request.form['form'] == 'Generate':
        val = request.form['inputState']
        df_ret=df[df['condition']==val]
        df_ret1=str(df_ret['County'].to_list())
        return render_template("low.html",mycontent2=df_ret1) 
    elif request.form['form'] == 'Send':
        val2 = request.form['userinput']
        y=function(val2)
        df_ret2=str(y)
        return render_template("low.html",mycontent1=df_ret2) 



In [ ]:
app.run(host='localhost', port=5005)

 * Serving Flask app "Interactive App" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5005/ (Press CTRL+C to quit)
127.0.0.1 - - [01/May/2022 22:17:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 22:17:21] "GET /jquery.captcha.basic.min.js HTTP/1.1" 404 -
127.0.0.1 - - [01/May/2022 22:17:21] "GET /jquery.captcha.basic.js HTTP/1.1" 404 -
127.0.0.1 - - [01/May/2022 22:17:21] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/May/2022 22:17:55] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 22:18:21] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 22:18:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 22:18:44] "GET /jquery.captcha.basic.min.js HTTP/1.1" 404 -
127.0.0.1 - - [01/May/2022 22:18:44] "GET /jquery.captcha.basic.js HTTP/1.1" 404 -
127.0.0.1 - - [01/May/2022 22:18:51] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 22:19:05] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 22:19:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 22:19:21] "GET /jquery.captcha.basic.min.js HTTP/1.1" 404 -
127.0.0.1 - - [01/

In [ ]:
# close the communication with the PostgreSQL
cur.close()